In [2]:
# install required packages and a compatible PyTorch build (CPU) to avoid the Config import error
%pip install -U pandas numpy nltk tqdm scikit-learn
%pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
    --------------------------------------- 0.3/11.3 MB ? eta -:--:--
   -- ------------------------------------- 0.8/11.3 MB 1.7 MB/s eta 0:00:07
   -- ------------------------------------- 0.8/11.3 MB 1.7 MB/s eta 0:00:07
   ---- ----------------------------------- 1.3/11.3 MB 1.4 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.3 MB 1.7 MB/s eta 0:00:06
   -------- ------------------------------- 2.4/11.3 MB 1.9 MB/s eta 0:00:05
   ----------- ---------------------------- 3.1/11.3 MB 2.1 MB/s eta 0:00:04
   --------

In [3]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [2]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')


[nltk_data] Downloading package stopwords to C:\Users\Dell
[nltk_data]     G15\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [4]:
class ArabicPreprocessor:
    """
    Arabic Text Preprocessing Pipeline
    Includes: Normalization, Tokenization, Stopwords Removal, Stemming
    """

    def __init__(self):
        self.arabic_stopwords = set([
            'في','من','إلى','هو','هي','ما','هذا','ذلك','التي','الذي',
            'و','أو','ثم','لكن','لو','إذا','لا','نعم','أن','إن',
            'مع','دون','خلال','بعد','قبل','عند','عن','بين','حول','ضد',
            'على','تحت','أمام','خلف','بجانب','كل','بعض','هنا','هناك',
            'كان','كانت','يكون','تكون','ال','الذين','حيث','كيف','ماذا'
        ])

        self.prefixes = ['ال','و','ف','ب','ك','ل','لل']
        self.suffixes = ['ها','ان','ات','ون','ين','يه','ه','ة','ي','ك','ت','ا','ن']
    def normalize_text(self, text):
        text = re.sub(r'[\u064B-\u065F]', '', text)
        text = re.sub(r'[أإآ]', 'ا', text)
        text = re.sub(r'[ؤ]', 'و', text)
        text = re.sub(r'[ئ]', 'ي', text)
        text = re.sub(r'[ة]', 'ه', text)
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    def tokenize(self, text):
        return text.split()

    def remove_stopwords(self, tokens):
        return [t for t in tokens if t not in self.arabic_stopwords]
    
    def simple_stemming(self, word):
        for prefix in self.prefixes:
            if word.startswith(prefix):
                word = word[len(prefix):]
                break
        for suffix in self.suffixes:
            if word.endswith(suffix) and len(word) > 3:
                word = word[:-len(suffix)]
                break
        return word

    def stem_tokens(self, tokens):
        return [self.simple_stemming(token) for token in tokens]
    
    def extract_features(self, text):
        words = text.split()
        return {
            'word_count': len(words),
            'char_count': len(text),
            'unique_words': len(set(words)),
            'lexical_diversity': len(set(words))/len(words) if words else 0
        }
    
    def preprocess(self, text):
        normalized = self.normalize_text(text)
        tokens = self.tokenize(normalized)
        filtered = self.remove_stopwords(tokens)
        stemmed = self.stem_tokens(filtered)
        stemmed = [t for t in stemmed if len(t) > 1]

        return {
            'original_text': text,
            'normalized_text': normalized,
            'tokens': tokens,
            'filtered_tokens': filtered,
            'stemmed_tokens': stemmed,
            'processed_text': ' '.join(stemmed),
            'features': self.extract_features(normalized)
        }



In [5]:
import os

csv_files = [f for f in os.listdir('.') if f.startswith('wikipedia_sports_dataset')]
csv_file = csv_files[-1]

df = pd.read_csv(csv_file, encoding='utf-8-sig')
df.head()


,Text,Score
0,سيكون موسم 2022–23 أول موسم يأخذ إجازة لمدة ست...,10.0
1,كرة القدمهيرياضة جماعيةتُلعب بين فريقين يتكون ...,10.0
2,حارس المرمىهو اللاعب الوحيد الذي يُسمح له أن ي...,10.0
3,وُضعت قوانين كرة القدم في إنجلترا سنة 1863 واس...,10.0
4,هناك نوعان من ألعاب كرة القدم وُثِّق وجودها تا...,10.0


In [6]:
preprocessor = ArabicPreprocessor()
results = []

for _, row in df.iterrows():
    processed = preprocessor.preprocess(row['Text'])
    processed['score'] = row['Score']
    results.append(processed)

print(f"Processed {len(results)} texts")


Processed 527 texts


In [7]:
processed_df = pd.DataFrame([{
    'Processed_Text': r['processed_text'],
    'Word_Count': r['features']['word_count'],
    'Lexical_Diversity': r['features']['lexical_diversity'],
    'Score': r['score']
} for r in results])

processed_df.head()
processed_df.to_csv('processed_arabic_texts.csv', index=False, encoding='utf-8-sig')


In [8]:
for i in range(3):
    print(f"\n--- Sample {i+1} ---")
    print("Original:", results[i]['original_text'][:150])
    print("Processed:", results[i]['processed_text'][:150])


--- Sample 1 ---
Original: سيكون موسم 2022–23 أول موسم يأخذ إجازة لمدة ستة أسابيع بين نوفمبر وديسمبر 2022 للسماح بأول كأس عالم شتوي،مع عودة مباريات يوم الملاكمة.
Processed: سيك موسم اول موسم ياخذ اجاز مده سته اسابيع نوفمبر ديسمبر لسماح اول اس عالم شتوي،مع عود مباري يوم ملاكم

--- Sample 2 ---
Original: كرة القدمهيرياضة جماعيةتُلعب بين فريقين يتكون كل منهما من أحد عشر لاعبًا تلعب بكرة مُكوَّرة. يلعب كرة القدم 250 مليون لاعب في أكثر من مائتيدولةحول الع
Processed: ره قدمهيرياض جماعيهتلعب ريق يتك منهم احد عشر اعب تلعب كره مكور يلعب ره قدم ملي اعب اكثر مايتيدولهحول عالم، ذلك تكونالرياضهالاكثر شعب انتشار عالمتلعب ر

--- Sample 3 ---
Original: حارس المرمىهو اللاعب الوحيد الذي يُسمح له أن يلمس الكرة بيديه أو ذراعيه، بشرط أن يكون داخلمنطقة الجزاء. يستخدم اللاعبون غير الحارس أرجلهم غالبًا في ال
Processed: حارس مرمىهو لاعب وحيد يسمح ان يلمس كره يد او ذراعيه، شرط ان داخلمنطق جزاء يستخدم لاعب غير حارس ارجلهم غالب تسديد او تمرير كره يمكنهم ايض استخدام راسهم


In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from tqdm import tqdm

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [10]:
df = pd.read_csv("processed_arabic_texts.csv", encoding="utf-8-sig")

texts = df["Processed_Text"].astype(str).values
scores = df["Score"].values.astype(np.float32)

scores = scores / 10.0  # normalize to [0,1]

print("Dataset size:", len(df))
df.head()

Dataset size: 527


,Processed_Text,Word_Count,Lexical_Diversity,Score
0,سيك موسم اول موسم ياخذ اجاز مده سته اسابيع نوف...,21,0.952381,10.0
1,ره قدمهيرياض جماعيهتلعب ريق يتك منهم احد عشر ا...,52,0.846154,10.0
2,حارس مرمىهو لاعب وحيد يسمح ان يلمس كره يد او ذ...,153,0.823529,10.0
3,ضعت قوان ره قدم انجلتر سنه استخدم اسم ره قدم ا...,76,0.855263,10.0
4,نوع عاب ره قدم ثق جود تاريخي اوروب هيايبسكسروس...,97,0.762887,10.0


In [11]:
from collections import Counter

# Tokenize
tokenized_texts = [text.split() for text in texts]

# Build vocabulary
counter = Counter(word for text in tokenized_texts for word in text)

word2idx = {"<PAD>": 0, "<UNK>": 1}
for word, freq in counter.items():
    if freq >= 2:
        word2idx[word] = len(word2idx)

vocab_size = len(word2idx)
max_len = 100

def encode(tokens):
    return [word2idx.get(w, word2idx["<UNK>"]) for w in tokens]

encoded = [encode(tokens) for tokens in tokenized_texts]

def pad(seq, max_len):
    return seq[:max_len] + [0] * max(0, max_len - len(seq))

X = torch.tensor([pad(seq, max_len) for seq in encoded])
y = torch.tensor(scores)

print("Vocab size:", vocab_size)
print("Input shape:", X.shape)


Vocab size: 2647
Input shape: torch.Size([527, 100])


In [12]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

In [14]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 32

train_loader = DataLoader(TextDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(TextDataset(X_val, y_val), batch_size=batch_size)
test_loader  = DataLoader(TextDataset(X_test, y_test), batch_size=batch_size)

In [15]:
class SequenceModel(nn.Module):
    def __init__(self, model_type, vocab_size, embed_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        if model_type == "RNN":
            self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True)
        elif model_type == "BiRNN":
            self.rnn = nn.RNN(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        elif model_type == "GRU":
            self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        elif model_type == "LSTM":
            self.rnn = nn.LSTM(embed_dim, hidden_dim, batch_first=True)

        self.bidirectional = model_type == "BiRNN"
        factor = 2 if self.bidirectional else 1

        self.fc = nn.Linear(hidden_dim * factor, 1)

    def forward(self, x):
        x = self.embedding(x)
        _, h = self.rnn(x)

        if isinstance(h, tuple):  # LSTM
            h = h[0]

        if self.bidirectional:
            h = torch.cat((h[-2], h[-1]), dim=1)
        else:
            h = h[-1]

        return self.fc(h).squeeze()


In [16]:
def train_model(model, train_loader, val_loader, epochs=10):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)

            optimizer.zero_grad()
            preds = model(Xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for Xb, yb in val_loader:
                Xb, yb = Xb.to(device), yb.to(device)
                val_loss += criterion(model(Xb), yb).item()

        print(f"Epoch {epoch+1} | Train MSE: {train_loss/len(train_loader):.4f} | Val MSE: {val_loss/len(val_loader):.4f}")


In [17]:
models = ["RNN", "BiRNN", "GRU", "LSTM"]
trained_models = {}

for name in models:
    print(f"\n🚀 Training {name}")
    model = SequenceModel(name, vocab_size, embed_dim=128, hidden_dim=128)
    train_model(model, train_loader, val_loader, epochs=10)
    trained_models[name] = model



🚀 Training RNN
Epoch 1 | Train MSE: 0.4928 | Val MSE: 0.2522
Epoch 2 | Train MSE: 0.0908 | Val MSE: 0.1339
Epoch 3 | Train MSE: 0.0502 | Val MSE: 0.1113
Epoch 4 | Train MSE: 0.0331 | Val MSE: 0.1047
Epoch 5 | Train MSE: 0.0286 | Val MSE: 0.1077
Epoch 6 | Train MSE: 0.0296 | Val MSE: 0.1058
Epoch 7 | Train MSE: 0.0296 | Val MSE: 0.1068
Epoch 8 | Train MSE: 0.0277 | Val MSE: 0.1070
Epoch 9 | Train MSE: 0.0275 | Val MSE: 0.1063
Epoch 10 | Train MSE: 0.0275 | Val MSE: 0.1038

🚀 Training BiRNN
Epoch 1 | Train MSE: 0.4210 | Val MSE: 0.1687
Epoch 2 | Train MSE: 0.0990 | Val MSE: 0.1120
Epoch 3 | Train MSE: 0.0423 | Val MSE: 0.0910
Epoch 4 | Train MSE: 0.0284 | Val MSE: 0.0726
Epoch 5 | Train MSE: 0.0186 | Val MSE: 0.0749
Epoch 6 | Train MSE: 0.0139 | Val MSE: 0.0712
Epoch 7 | Train MSE: 0.0111 | Val MSE: 0.0717
Epoch 8 | Train MSE: 0.0114 | Val MSE: 0.0731
Epoch 9 | Train MSE: 0.0090 | Val MSE: 0.0708
Epoch 10 | Train MSE: 0.0068 | Val MSE: 0.0729

🚀 Training GRU
Epoch 1 | Train MSE: 0.4533 

In [18]:
results = []

for name, model in trained_models.items():
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for Xb, yb in test_loader:
            Xb = Xb.to(device)
            preds.extend(model(Xb).cpu().numpy())
            targets.extend(yb.numpy())

    mse = mean_squared_error(targets, preds)
    mae = mean_absolute_error(targets, preds)
    r2 = r2_score(targets, preds)

    results.append({
        "Model": name,
        "MSE": mse,
        "MAE": mae,
        "R2": r2
    })

results_df = pd.DataFrame(results)
results_df


,Model,MSE,MAE,R2
0,RNN,0.067711,0.146346,-0.232917
1,BiRNN,0.067592,0.180930,-0.230743
2,GRU,0.069793,0.153937,-0.270820
3,LSTM,0.064664,0.130856,-0.177427
